In [ ]:
library(miloR)
library(igraph)
library(BiocParallel)
library(SingleCellExperiment)
library(Matrix)
library(dplyr)

library(BiocParallel)
library(miloR)
library(ggplot2)

Loading required package: edgeR

Loading required package: limma


Attaching package: ‘igraph’


The following object is masked from ‘package:miloR’:

    graph


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    c

In [ ]:
load('../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007__milo_prep.RData')
adata_r

In [ ]:
mylo <- Milo(adata_r)
milo_graph <- buildFromAdjacency(knn_adjacencyx, k=50, is.binary=TRUE)
miloR::graph(mylo) <- miloR::graph(milo_graph)

In [ ]:
dim(reducedDims(mylo)[['X_scVI']])

In [ ]:
mylo <- buildGraph(mylo, k=50, d=10, reduced.dim="X_scVI",  BPPARAM = MulticoreParam(workers = 24, progressbar = TRUE))
mylo <- makeNhoods(mylo, prop = 0.3, k=50, d=10, reduced_dim="X_scVI", refined=TRUE, refinement_scheme="graph")

In [ ]:
saveRDS(mylo, '../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_makeNhoods.RDS') 

In [ ]:
mylo <- countCells(mylo, meta.data = data.frame(colData(mylo)), samples="pica_id") 

In [ ]:
mylo <- calcNhoodDistance(mylo, d=10, reduced.dim = 'X_scVI', use.assay = 'counts')

In [ ]:
saveRDS(mylo, '../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_calcNhoodDistance.RDS') 

In [ ]:
mylo <- readRDS('../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_calcNhoodDistance.RDS')

In [ ]:
mylo <- buildNhoodGraph(mylo)

In [ ]:
saveRDS(mylo, '../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_buildNhoodDistance.RDS')

In [ ]:
metadata = data.frame(colData(mylo))
colnames(metadata)

In [ ]:
# Set the reference level for all factor cols
metadata$HealthStatus <- factor(metadata$condition, levels=c("OSA","OME"))
metadata$Age <- as.numeric(metadata$ages)
metadata$Age2 <- metadata$Age^2
metadata$Gender <- factor(metadata$gender,levels=c("F","M"))
metadata$Sample <- as.factor(metadata$batch_merged)

In [ ]:
# Subset the metadata
metadata <- distinct(metadata[,c('HealthStatus', 'Age', 'Age2', 'Gender', 'Sample')], .keep_all = TRUE)
metadata

In [ ]:
rownames(metadata) <- metadata$Sample
metadata

In [ ]:
table(metadata$Age)

## Testing

In [ ]:
x.model <- model.matrix(~ HealthStatus + Age + Age2 + Gender + HealthStatus:Gender + HealthStatus:Age + HealthStatus:Age2 + Age:Gender + Gender:Age2, data=metadata)
dge <- DGEList(counts=nhoodCounts(mylo), cell.sizes=colSums(nhoodCounts(mylo)))
dge <- estimateDisp(dge, x.model)

In [ ]:
it <- glmQLFit(dge, x.model, robust=TRUE, legacy=TRUE)

In [ ]:
out <- list()

for (coef in c("HealthStatusOME","Age","Age2","GenderM","HealthStatusOME:GenderM","HealthStatusOME:Age","HealthStatusOME:Age2","Age:GenderM","Age2:GenderM")){
    res <- as.data.frame(topTags(glmQLFTest(fit, coef=coef), sort.by='none', n=Inf))
    res$Nhood <- as.numeric(rownames(res))
    mod.spatialfdr <- graphSpatialFDR(x.nhoods=nhoods(mylo),
                                      graph=miloR::graph(mylo),
                                      weighting="graph-overlap",
                                      k=mylo@.k,
                                      pvalues=res[order(res$Nhood), ]$PValue,
                                      indices=nhoodIndex(mylo),
                                      distances=nhoodDistances(mylo),
                                      reduced.dimensions=reducedDim(mylo, "X_scVI"))
    res$SpatialFDR[order(res$Nhood)] <- mod.spatialfdr
    out[[coef]] <- res
}

In [ ]:
save(mylo, out, file= "../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_out.RData")

In [ ]:
load("../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_out.RData")

In [ ]:
out <- bplapply(out, function(x){
    # Assign cell types to neighborhoods
    x <- annotateNhoods(mylo, x, coldata_col = "cell_type")
    # Heterogenous neighborhoods are re-annotaed as Mixed
    x$celltype <- ifelse(x$celltype_fraction < 0.7, "Mixed", x$celltype)
    x$celltype <- as.factor(x$celltype)
    return(x)
}, BPPARAM=MulticoreParam(progressbar=TRUE, workers=9))

In [ ]:
ave(mylo, out, file= "../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_out2.RData")

In [ ]:
load("../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_out2.RData")

In [ ]:
out[[9]] %>% # manually check the result for each contrast 
  arrange(SpatialFDR) %>%
  head() 